# Gloss passage against Hyginus, *Fabulae*

> *Add link here to all the caveats...*


### How to use this notebook

1. First, run step 1 (e.g., by selecting the cell labelled **Step 1: load everything** and choosing "Run all below" from the "Cell" menu).  This will be slow, and your mileage may vary depending on how well your connection to different resources on the internet happens to be performing just then.
2. Just below the cell labelled **Step 2: highlight this text for glossing**, fill in between quotation marks an argument to the function `highlightThis`.  E.g., to highlight vocabulary not covered `duco` in Hyginus, you could edit that cell to read


    `val highlightThis = "Hercules Busiridem interfecit."`
    
    

Then run the cell (e.g., by selecting it, and choosing "Run cells" from the "Cell" menu).

# Step 2: highlight this text for glossing

In [43]:
val highlightThis = "Pater filiam suam Herculi dedit."
val vocabUnit = 1
highlight(highlightThis, vocabUnit)


Highlight with vocabulary from unit 1 Unit 1 includes 36 vocabulary items. Text to highlight: Pater filiam suam Herculi dedit. Highlighted view Key In vocab list 
 Not analyzed 
 Mutiple analyses Pater filiam suam Herculi dedit.

highlightThis: String = "Pater filiam suam Herculi dedit."
vocabUnit: Int = 1

# Step 1: load everything


In [ ]:
// set up notebook to find repository
val personalRepo = coursierapi.MavenRepository.of("https://dl.bintray.com/neelsmith/maven")
interp.repositories() ++= Seq(personalRepo)

In [ ]:
// ivy imports
import $ivy.`edu.holycross.shot::latincorpus:7.0.0-pr5`
import $ivy.`edu.holycross.shot.mid::orthography:2.1.0`
import $ivy.`edu.holycross.shot::latphone:3.0.0`
import $ivy.`edu.holycross.shot.cite::xcite:4.3.0`
import $ivy.`edu.holycross.shot::ohco2:10.20.4`

In [ ]:
import edu.holycross.shot.latincorpus._
import edu.holycross.shot.latin._
import edu.holycross.shot.mid.orthography._
import edu.holycross.shot.cite._
import edu.holycross.shot.ohco2._
import scala.io.Source

In [ ]:
val hyginusUrl = "https://raw.githubusercontent.com/LinguaLatina/analysis/master/data/hyginus/hyginus-latc.cex"
val hyginus = LatinCorpus.fromUrl(hyginusUrl)

In [ ]:
// Settings:
val lat23orthogaphy: MidOrthography = Latin23Alphabet
val fstUrl = "https://raw.githubusercontent.com/LinguaLatina/analysis/master/data/hyginus/hyginus-fst.txt"
val fstLines = Source.fromURL(fstUrl).getLines.toVector


val vocabFiles : Map[Int, String] = Map(
  1 -> "https://raw.githubusercontent.com/LinguaLatina/textbook/master/vocablists/01-nouns-adjs-pron.cex"
)

def vocabForUnit(vocabUnit: Int): Vector[String] = {
  val lines = Source.fromURL(vocabFiles(vocabUnit))
  val lexemeIds = lines.getLines.toVector.tail.filter(_.nonEmpty).map( ln => {
    val columns = ln.split("#")
    val idParts = columns.head.split(":")
    idParts.head
  })
  lexemeIds
}


## CSS styling

If you want to define your own visual styling for highlighting ambiguous tokens, unanalyzed tokens, and tokens from the chapter vocabulary, you can edit the following cell, then rerun both of the cells below this (e.g., select this cell and choose "Run all below" from the "Cell" menu). 

Your new styling will then be used the next time you run **Step 2: highlight this text for glossing**.


In [ ]:
// CSS definitions

// CSS styling for tokens with multiple analyses
val ambiguousCss = "text-decoration-line: underline; text-decoration-style: wavy; text-decoration-color: silver;"
// CSS styling for tokens with no analyses
val unanalyzedCss = "text-decoration-line: underline; text-decoration-style: double; text-decoration-color:rgb(159, 69, 17);"
// CSS styling for tokens in vocabulary list for this unit
val vocabCss = "color:rgb(24, 23, 162);"

In [ ]:
// The action:
 val bullets = Vector(
   "<li style=\"" + vocabCss + "\">In vocab list</li>",
   "<li  style=\"" + unanalyzedCss + "\">Not analyzed</li>",
   "<li  style=\"" + ambiguousCss + "\">Mutiple analyses</li>"
   )
val styleKey = "<h3>Key</h3><ul>" + bullets.mkString("\n") + "</ul>"
       


def highlight(txt: String, vocabUnit: Int) = {
    if (! vocabFiles.keySet.contains(vocabUnit)) {
      Html("Sorry, but currently only have vocabulary lists for these units:  " + vocabFiles.keySet.toVector.sorted.mkString(", "))

    } else {
      try {
        val vocab = vocabForUnit(vocabUnit)
        val highlighter = LexemesHighlighter(vocab, vocabCss)
        // make an ohco2 Corpus in order to tokenize text.
        val fakeUrn = CtsUrn("urn:cts:fake:exercise.v1:1")
        val citable = CitableNode(fakeUrn, txt)
        val exercise = Corpus(Vector(citable))
        val latc = LatinCorpus.fromFstLines(exercise,lat23orthogaphy, fstLines, strict=false)

        val formatted = StringFormatter.tokensLexemeStyled(
          latc.tokens,
          highlighter,
          vocab,
          formAmbiguityStyle = ambiguousCss, 
          lexicalAmbiguityStyle = "", 
          unanalyzedStyle = unanalyzedCss
        )        
        
        val header = s"<h2>Highlight with vocabulary from unit ${vocabUnit}</h2>"
        val nb = s"<p>Unit ${vocabUnit} includes <strong>${vocab.size}</strong> vocabulary items."
        val prefatoryPara = "<p>Text to highlight: <em>" + txt + "</em></p>"
        
        Html(header + nb + prefatoryPara +
             s"<hr/><h2>Highlighted view</h2>${styleKey}<p>${formatted}")
        
        } catch {
         case nope: java.util.NoSuchElementException => {
         Html("<p><em>" + txt + "</em> : something went wrong. :-(</p>")
         }
       }
      }
}